<a href="https://colab.research.google.com/github/hectorjimenez12/FONDEF_2023_Debris_Flow/blob/main/01Experimentos/03_Results_FiguresBE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install --force-reinstall -v "scikit-learn==1.5.1"

In [5]:
import sys
sys.path.append('..')
import pdb
import copy
import matplotlib.pyplot as plt
import joblib
import sklearn
import numpy as np
import random
SEED = 4376
random.seed(SEED)
np.random.seed(SEED)

In [13]:
from source.BFS_Model import *
from source.utils import *

In [10]:
columns_required = ['max_precip', 'total_precip', 'melton_index', 'mean_slope',
                    'california_tc', 'area', 'river_slope', 'main_river_slope',
                    'drainage_density', 'long_outlet', 'vel_ms']

# 1. Modelos

In [12]:

rootfolderBE = '../models/minmax_nas'
fileseedsBE = os.listdir(rootfolderBE)
len(fileseedsBE)
seeds = [i.split('ModelResults')[1].split('.pickle')[0] for i in fileseedsBE[0:50]]

seeds_BE    = []
seeds_fail  = []
modelsObj   = []
for k, i in enumerate(fileseedsBE):
  if k%10 == 0:
    print(k) 
  try:
    a = joblib.load( rootfolderBE + '/' + i )
    seeds_BE.append(int(seeds[k]))
    modelsObj.append(a)
  except:
    seeds_fail.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


# 2. Ver resultados

In [ ]:
XConditions = pd.read_csv('X_example.csv') # No se cambia
#XConditions['main_river_slope'] = 40
#XConditions['river_slope']      = 40

modelsPredictions = {}
for i, iseed in enumerate(seeds_BE):
    if i%20 == 0:
        print(i)

    classmodel, resmodel             = get_model_and_results_byIndex(index = i,
                                                             model = 'MultiLayerPerceptron') #SupportVectorClassifier
    test_best, feat_best, iter_best  = get_select_model(resmodel  , ind_sel = 0, treshold = 0.02 )

    data_best_iter = list(resmodel.values())[iter_best]
    model_best, feat_best = data_best_iter['model'] ,data_best_iter['features']

    modelsPredictions[iseed] = {}
    
    Xmap, Ymap, scalerObject = Scaling(X, y, iseed)
    Xtest = Scaling_NewData(XConditions,scalerObject)

    y_pred = Prediction(model_best, feat_best, Xtest )
    modelsPredictions[iseed] = y_pred

Plots

In [ ]:
DFpredictions = pd.DataFrame.from_dict( modelsPredictions,orient = 'index' )
#DFpredictions = pd.DataFrame(modelsPredictions)
fig, ax = plt.subplots(figsize = (10,5))
DFpredictions.boxplot(ax = ax)
plt.xlabel('Quebrada')
plt.ylabel('Probabilidad')
plt.savefig('Figura.png')
DFpredictions.to_csv('Predicciones.csv')

plt.figure(figsize = (14,6))
X.boxplot()
for i,icol in enumerate(XConditions.columns):
	plt.scatter([i+1]*len(XConditions), XConditions[icol],color = 'red', zorder = 3)
plt.savefig('Figura2.png')